In [8]:
import os
import pandas as pd

In [9]:
pairs_path = '../data/lfw/pairs.txt'
print(os.path.isfile(pairs_path))

True


In [48]:
data = pd.read_csv(pairs_path, sep = '\t',names = ['1','2','3','4'], index_col = False)
data.head()

,1,2,3,4
0,10,300,NaN,NaN
1,Abel_Pacheco,1,4,NaN
2,Akhmed_Zakayev,1,3,NaN
3,Akhmed_Zakayev,2,3,NaN
4,Amber_Tamblyn,1,2,NaN


In [51]:
data2 = data[:100]
data2.head()

,1,2,3,4
0,10,300,NaN,NaN
1,Abel_Pacheco,1,4,NaN
2,Akhmed_Zakayev,1,3,NaN
3,Akhmed_Zakayev,2,3,NaN
4,Amber_Tamblyn,1,2,NaN


In [63]:
data2.to_csv('../data/lfw/small_pairs.txt', index = False, header = False, sep = '\t')

# image to .bin
- https://github.com/deepinsight/insightface/blob/master/src/data/lfw2pack.py

In [64]:
import cv2
import pickle
import numpy as np
import sys
from scipy import misc
import os
import tensorflow as tf
from scipy.io import loadmat
from easydict import EasyDict as edict
import os
import json
import numpy as np

# sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'align'))
#sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'common'))

# aligned된 사진으로 수행하기

## lfw.py

In [65]:
def add_extension(path):
    if os.path.exists(path+'.jpg'):
        return path+'.jpg'
    elif os.path.exists(path+'.png'):
        return path+'.png'
    else:
        raise RuntimeError('No file "%s" with extension png or jpg.' % path)
        
def read_pairs(pairs_filename):
    pairs = []
    with open(pairs_filename, 'r') as f:
        for line in f.readlines()[1:]:
            pair = line.strip().split()
            pairs.append(pair)
    return np.array(pairs)

def get_paths(lfw_dir, pairs):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    print('len(pairs)', len(pairs))
    for i,pair in enumerate(pairs):
        print(i, pair)
        if len(pair) == 3:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[2])))
            issame = True
        elif len(pair) == 4:
            path0 = add_extension(os.path.join(lfw_dir, pair[0], pair[0] + '_' + '%04d' % int(pair[1])))
            path1 = add_extension(os.path.join(lfw_dir, pair[2], pair[2] + '_' + '%04d' % int(pair[3])))
            issame = False
        if os.path.exists(path0) and os.path.exists(path1):    # Only add the pair if both paths exist
            path_list += (path0,path1)
            issame_list.append(issame)
        else:
            nrof_skipped_pairs += 1
    if nrof_skipped_pairs>0:
        print('Skipped %d image pairs' % nrof_skipped_pairs)
    return path_list, issame_list

## lfw2pack.py

In [70]:
args = edict()
args.data_dir = '../data/lfw/lfw/'
args.image_size = '112,96'
args.output = '../data/lfw/small_lfw.bin'
args.model_path = './mtcnn_weights'
print(os.path.isdir(args.data_dir))

True


In [71]:
lfw_dir = args.data_dir
image_size = [int(x) for x in args.image_size.split(',')]
lfw_pairs = read_pairs(os.path.join(lfw_dir, 'small_pairs.txt'))
print(lfw_pairs.shape)

(99, 3)


In [72]:
lfw_paths, issame_list = get_paths(lfw_dir, lfw_pairs)
lfw_bins = []

len(pairs) 99
0 ['Abel_Pacheco' '1' '4']
1 ['Akhmed_Zakayev' '1' '3']
2 ['Akhmed_Zakayev' '2' '3']
3 ['Amber_Tamblyn' '1' '2']
4 ['Anders_Fogh_Rasmussen' '1' '3']
5 ['Anders_Fogh_Rasmussen' '1' '4']
6 ['Angela_Bassett' '1' '5']
7 ['Angela_Bassett' '2' '5']
8 ['Angela_Bassett' '3' '4']
9 ['Ann_Veneman' '3' '5']
10 ['Ann_Veneman' '6' '10']
11 ['Ann_Veneman' '10' '11']
12 ['Anthony_Fauci' '1' '2']
13 ['Antony_Leung' '1' '2']
14 ['Antony_Leung' '2' '3']
15 ['Anwar_Ibrahim' '1' '2']
16 ['Augusto_Pinochet' '1' '2']
17 ['Barbara_Brezigar' '1' '2']
18 ['Benjamin_Netanyahu' '1' '4']
19 ['Benjamin_Netanyahu' '4' '5']
20 ['Bernard_Law' '2' '3']
21 ['Bernard_Law' '3' '4']
22 ['Bertrand_Bonello' '1' '2']
23 ['Bill_Frist' '2' '9']
24 ['Bill_Frist' '4' '5']
25 ['Bob_Graham' '2' '4']
26 ['Bob_Graham' '3' '6']
27 ['Bob_Graham' '4' '6']
28 ['Bob_Graham' '5' '6']
29 ['Boris_Becker' '2' '6']
30 ['Brad_Johnson' '1' '3']
31 ['Brad_Johnson' '2' '3']
32 ['Brad_Johnson' '2' '4']
33 ['Brian_Griese' '1' '2']
34 

In [73]:
i = 0
for path in lfw_paths:
  with open(path, 'rb') as fin:
    _bin = fin.read()
    lfw_bins.append(_bin)
    #img = mx.image.imdecode(_bin)
    #img = nd.transpose(img, axes=(2, 0, 1))
    #lfw_data[i][:] = img
    i+=1
    if i%1000==0:
      print('loading lfw', i)

with open(args.output, 'wb') as f:
  pickle.dump((lfw_bins, issame_list), f, protocol=pickle.HIGHEST_PROTOCOL)
